In [4]:
from random import sample
from numpy import mean,average, std

path_dist = lambda point, limit: abs(point[0]/limit[0]-point[1]/limit[1])

class ant:
    def __init__(self, x, y, grid):
        self.position = [x,y]
        self.grid = grid
        self.max_dist = path_dist(self.position,grid.get_extent())
        if tuple(self.position)==grid.get_extent:
            self.finished=True
        else:
            self.finished=False
    def get_position(self):
        return self.position
    def get_x(self):
        return self.position[0]
    def get_y(self):
        return self.position[1]
    def get_d(self):
        return self.max_dist
    def x_step(self):
        self.position[0]+=1
    def y_step(self):
        self.position[1]+=1
    def move(self):
        if self.get_x()<self.grid.get_xlim():
            if self.get_y()<self.grid.get_ylim():
                # "normal random move"
                direction = sample([0,1],1)[0]
                self.position[direction]+=1
            else:
                # hit y_lim, move x
                self.x_step()
        elif self.get_y()<self.grid.get_ylim():
            # hit x_lim but not y_lim, move y
            self.y_step()
        # check again if in corner
        if self.get_x()==self.grid.get_xlim() and self.get_y()==self.grid.get_ylim():
            #print("Reached northeast corner")
            self.finished=True
        # update distance
        self.max_dist = max(self.max_dist, path_dist(self.position,self.grid.get_extent()))
        return self.position
    
class grid:
    def __init__(self,m,n):
        self.extent = (m,n)
    def get_xlim(self):
        return self.extent[0]
    def get_ylim(self):
        return self.extent[1]
    def get_extent(self):
        return self.extent

def ant_run(m,n):
    chessboard = grid(m,n)
    thisAnt=ant(0,0,chessboard)
    
    # move it until it gets there
    while not thisAnt.finished:
        thisAnt.move()
    
    # return the maximum distance of the move
    return thisAnt.get_d()

def simulate(m,n):
    # run until delta is sufficiently small to get all the sig figs
    d_0 = ant_run(m,n)
    ds = [d_0]
    d_mean = d_0
    # get to the point where the mean doesn't change much between runs.
    for N in range(1,10000000):
        # get another run
        d_i = ant_run(m,n)
        # add this run to the register
        ds += [d_i]
        # get the new mean
        d_mean_i = average([d_mean,d_i],weights=[N,1])
        # get the delta and update N
        delta = d_mean_i-d_mean
        # update the running mean
        d_mean = d_mean_i
        if N%1000000 == 0:
            print(N, delta, d_mean, std(ds))
    # once delta gets small enough, return N, mean and standard deviation
    return N, delta, d_mean, ds, std(ds)
        


In [5]:
m23n31 = simulate(23,31)

1000000 8.563735898192704e-08 0.35335282336505 0.13697040925094978
2000000 -5.3257162013764514e-08 0.35335864379816917 0.1369432965976226
3000000 1.2645176239445277e-08 0.35336881909764056 0.13700285465044165
4000000 9.844176640516622e-09 0.353330165876001 0.13702112283782025
5000000 -3.7849427092506716e-08 0.3533425071726335 0.13698657595749222
6000000 3.788241537128201e-08 0.3533506690196598 0.13697227793398203
7000000 -2.282753852611563e-08 0.353341156698649 0.1369810816870318
8000000 -5.16265752370515e-10 0.353358737742007 0.13697462538219493
9000000 1.4501779999065434e-08 0.3533549474977361 0.13697259894775335


In [6]:
m7n11 = simulate(7,11)

1000000 -1.5525018237649846e-07 0.5188865460491395 0.18186423448357417
2000000 -8.3998795585849e-08 0.5186469419759859 0.18168109993362808
3000000 -4.306088741135028e-08 0.5187930521776523 0.18180849891980824
4000000 -1.606216237792779e-08 0.5187941040679562 0.1818265473265081
5000000 5.988059170736193e-08 0.5187788598836732 0.18179179029646012
6000000 4.123734020922143e-08 0.5188097252250995 0.18180689822507898
7000000 2.9774991472386603e-08 0.5188477867388449 0.18182074217888017
8000000 -2.7522058854145826e-08 0.5188777695633984 0.18180132990283554
9000000 -8.588277422560964e-09 0.5188529380241724 0.1818169237906158


In [10]:
prior=[d for d in m23n31[3] if d>0.2]
actual=[d for d in prior if d>0.6]
print(len(actual)/len(prior))

0.06318815017291772


In [11]:
prior=[d for d in m7n11[3] if d>0.2]
actual=[d for d in prior if d>0.6]
print(len(actual)/len(prior))

0.3282538209840449


23,31: 63000000 2.0727117244767612e-09 0.35329013285949273 0.13694944570100703

7,11: 293000000 2.2362933727038126e-10 0.5188921989732688 0.18177564236980565